In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-demo/pytorch_model_distilbert-base-vi-cased_dtv3.bin
/kaggle/input/idioms-data/README.md
/kaggle/input/idioms-data/ViID_dict.txt
/kaggle/input/idioms-data/data_ver2/dev.json
/kaggle/input/idioms-data/data_ver2/train.json
/kaggle/input/idioms-data/data_ver2/test.json
/kaggle/input/idioms-data/data_ver1/dev.json
/kaggle/input/idioms-data/data_ver1/train.json
/kaggle/input/idioms-data/data_ver1/test.json
/kaggle/input/idioms-data/pytorch_pretrained_bert/modeling.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/file_utils.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/tokenization.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/convert_tf_checkpoint_to_pytorch.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/optimization.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/__init__.py
/kaggle/input/idioms-data/pytorch_pretrained_bert/__main__.py


In [2]:
!git clone https://huggingface.co/Geotrend/distilbert-base-vi-cased

Cloning into 'distilbert-base-vi-cased'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 10
Unpacking objects: 100% (14/14), 58.66 KiB | 6.52 MiB/s, done.
Filtering content: 100% (2/2), 434.86 MiB | 123.62 MiB/s, done.


In [3]:
%cd '/kaggle/input/idioms-data'

/kaggle/input/idioms-data


In [5]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

idiom_vocab = eval(open('/kaggle/input/idioms-data/ViID_dict.txt').readline())
idiom_vocab = {each: i for i, each in enumerate(idiom_vocab)}

In [4]:
import argparse
import collections
import logging
import json
import math
import os
import random
import pickle
from tqdm import tqdm, trange
import re
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch import nn
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer

from pytorch_pretrained_bert.tokenization import whitespace_tokenize, BasicTokenizer, BertTokenizer
from pytorch_pretrained_bert.modeling import PreTrainedBertModel, BertModel, BertConfig, BertForMultipleChoice
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [6]:
class ViidExample(object):
    def __init__(self,
                 tag,
                 context,
                 options,
                 label=None):
        self.tag = tag
        self.context = context
        self.options = options
        self.label = label

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "tag: %s" % (self.tag)
        s += ", context: %s" % (self.context)
        s += ", options: [%s]" % (", ".join(self.options))
        if self.label is not None:
            s += ", answer: %s" % self.options[self.label]
        return s

In [7]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 tag,
                 input_ids,
                 input_mask,
                 segment_ids,
                 option_ids,
                 position,
                 label):
        self.tag = tag # use int number
        self.input_ids = input_ids # [max_seq_length]
        self.input_mask = input_mask # [max_seq_length]
        self.segment_ids = segment_ids # [max_seq_length]
        self.option_ids = option_ids # [num_choices]
        self.position = position
        self.label = label

In [8]:
max_seq_length = 256
doc_stride = 128
max_query_length = 128
do_train = True
do_predict = True
train_batch_size = 32
predict_batch_size = 64
learning_rate = 2*10**-5
num_train_epochs = 10
warmup_proportion = 0.1
n_best_size = 20
max_answer_length = 32
verbose_logging = False

no_cuda = False
seed = 42
gradient_accumulation_steps = 1
do_lower_case = True
local_rank = -1
fp16 = False
loss_scale = 0

In [9]:
if local_rank == -1 or no_cuda:
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
    device, n_gpu, bool(local_rank != -1), fp16))

if gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                        gradient_accumulation_steps))
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

In [10]:
def get_choice(idiom):
      return ''.join([s.lower() for s in idiom])

In [11]:
class BertForCloze(PreTrainedBertModel):
    """BERT model for multiple choice tasks.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.

    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_choices`: the number of classes for the classifier. Default = 2.

    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length]
            with the token types indices selected in [0, 1]. Type 0 corresponds to a `sentence A`
            and type 1 corresponds to a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `option_ids`: a torch.LongTensor of shape [batch_size, num_choices]
        `positions`: a torch.LongTensor of shape [batch_size]
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_choices].

    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    """
    def __init__(self, config, num_choices):
        super(BertForCloze, self).__init__(config)
        self.num_choices = num_choices
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 1)
        self.idiom_embedding = nn.Embedding(len(idiom_vocab), config.hidden_size)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, option_ids, token_type_ids, attention_mask, positions, labels=None):

        encoded_layer, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        blank_states = encoded_layer[[i for i in range(len(positions))], positions] # [batch, hidden_state]

        encoded_options = self.idiom_embedding(option_ids)
        multiply_result = torch.einsum('abc,ac->abc', encoded_options, blank_states)

        pooled_output = self.dropout(multiply_result)
        logits = self.classifier(pooled_output)
        reshaped_logits = logits.view(-1, self.num_choices)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)
            return loss
        else:
            return reshaped_logits

In [12]:
def read_query(query, choices, arr):
    if arr == 0:
        if len(re.findall(r'\[BLANK\d+\]', query)) == 1:
            tag = '[' + ''.join(re.findall("BLANK1", query)) + ']'
            tmp_context = query

            label = None
            options = choices

            return ViidExample(
                tag=tag,
                context=tmp_context,
                options=choices,
                label=label
            )
        else: 
            examples = []
            tags = re.findall(r'\[BLANK\d+\]', query)
            # print(tags)
            # return
            options = choices
            idx = 0
            for tag in tags:
                tmp_context = query
                for other_tag in tags:
                    if other_tag != tag:
                        tmp_context = tmp_context.replace(other_tag, "[UNK]")
                label = None
                examples.append(ViidExample(
                    tag=tag,
                    context=tmp_context,
                    options=options,
                    label=label
                ))
            return examples
    else: 
        examples = []
        for idx, q in enumerate(query):
            tags = re.findall(r'\[BLANK\d+\]', q)
            options = choices[idx]
            idx = 0
            for tag in tags:
                tmp_context = q
                for other_tag in tags:
                    if other_tag != tag:
                        tmp_context = tmp_context.replace(other_tag, "[UNK]")
                label = None
                examples.append(ViidExample(
                    tag=tag,
                    context=tmp_context,
                    options=options,
                    label=label
                ))
        return examples
def convert_query_to_feature(query, tokenizer, max_seq_length, is_list):
    if is_list == 0:
        label = None
        if query.label is not None:
            label = query.label

        tag = query.tag
        context = query.context

        parts = context.split(tag)

        assert len(parts) == 2
        before_part = tokenizer.tokenize(parts[0]) if len(parts[0]) > 0 else []
        after_part = tokenizer.tokenize(parts[1]) if len(parts[1]) > 0 else []


        half_length = int(max_seq_length / 2)
        if len(before_part) < half_length: # cut at tail
            st = 0
            ed = min(len(before_part) + 1 + len(after_part), max_seq_length - 2)
        elif len(after_part) < half_length: # cut at head
            ed = len(before_part) + 1 + len(after_part)
            st = max(0, ed - (max_seq_length - 2))
        else: # cut at both sides
            st = len(before_part) + 3 - half_length
            ed = len(before_part) + 1 + half_length

        option_ids = [idiom_vocab[get_choice(each)] for each in query.options]

        tokens = before_part + ["[MASK]"] + after_part
        tokens = ["[CLS]"] + tokens[st:ed] + ["[SEP]"]

        position = tokens.index("[MASK]")

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        segment_ids = [0] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        return InputFeatures(
            tag=int(tag[6: -1]),
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids,
            option_ids=option_ids,
            position=position,
            label=label
        )
    else:
        features = []
        for example in query:

            label = None
            if example.label is not None:
                label = example.label

            tag = example.tag
            context = example.context
            parts = context.split(tag)
            # print(parts)
            assert len(parts) == 2
            before_part = tokenizer.tokenize(parts[0]) if len(parts[0]) > 0 else []
            after_part = tokenizer.tokenize(parts[1]) if len(parts[1]) > 0 else []

            half_length = int(max_seq_length / 2)
            if len(before_part) < half_length: # cut at tail
                st = 0
                ed = min(len(before_part) + 1 + len(after_part), max_seq_length - 2)
            elif len(after_part) < half_length: # cut at head
                ed = len(before_part) + 1 + len(after_part)
                st = max(0, ed - (max_seq_length - 2))
            else: # cut at both sides
                st = len(before_part) + 3 - half_length
                ed = len(before_part) + 1 + half_length

            option_ids = [int(idiom_vocab[get_choice(each)]) for each in example.options]
            # print(example.options)
            tokens = before_part + ["[MASK]"] + after_part
            tokens = ["[CLS]"] + tokens[st:ed] + ["[SEP]"]
            position = tokens.index("[MASK]")
            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)
            segment_ids = [0] * len(input_ids)

            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            features.append(InputFeatures(
                tag=int(tag[6: -1]),
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                option_ids=option_ids,
                position=position,
                label=label
            ))

        return features

In [13]:
def get_feature_for_pred(query, choices, is_list, arr):
    input = read_query(query, choices, arr)
    feature = convert_query_to_feature(input, tokenizer, max_seq_length, is_list)
    if is_list == 0:
        input_ids = torch.tensor([feature.input_ids], dtype=torch.long)
        input_mask = torch.tensor([feature.input_mask], dtype=torch.long)
        segment_ids = torch.tensor([feature.segment_ids], dtype=torch.long)
        option_ids = torch.tensor([feature.option_ids], dtype=torch.long)
        positions = torch.tensor([feature.position], dtype=torch.long)
        tags= torch.tensor([feature.tag], dtype=torch.long)
    
        # eval_data = TensorDataset(input_ids, input_mask, segment_ids, option_ids, positions, tags)

        model.eval()

        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        option_ids = option_ids.to(device)
        positions = positions.to(device)

        with torch.no_grad():
            batch_logits = model(input_ids, option_ids, segment_ids, input_mask, positions)
        answer = None
        for i, tag in enumerate(tags):
            logits = batch_logits[i].detach().cpu().numpy()
            ans = np.argmax(logits)
            answer = ans

        return answer
    else:
        all_input_ids = torch.tensor([f.input_ids for f in feature], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in feature], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in feature], dtype=torch.long)
        all_option_ids = torch.tensor([f.option_ids for f in feature], dtype=torch.long)
        all_positions = torch.tensor([f.position for f in feature], dtype=torch.long)
        all_tags= torch.tensor([f.tag for f in  feature],dtype=torch.long)
        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_option_ids, all_positions, all_tags)
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=predict_batch_size)

        model.eval()
        all_results = {}
        logger.info("Start evaluating")
        count = 0
        for input_ids, input_mask, segment_ids, option_ids, positions, tags in \
                tqdm(eval_dataloader, desc="Evaluating",disable=None):
            if len(all_results) % 1000 == 0:
                logger.info("Processing example: %d" % (len(all_results)))
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            option_ids = option_ids.to(device)
            positions = positions.to(device)

            with torch.no_grad():
                batch_logits = model(input_ids, option_ids, segment_ids, input_mask, positions)
            # print(len(tags))
            for i, tag in enumerate(tags):
                logits = batch_logits[i].detach().cpu().numpy()
                ans = np.argmax(logits)
                all_results[str(count)] = ans
                count += 1
        return all_results

In [14]:
output_model_file = '/kaggle/input/model-demo/pytorch_model_distilbert-base-vi-cased_dtv3.bin'
pretrained_model = 'Geotrend/distilbert-base-vi-cased'
bert_config_file = '/kaggle/working/distilbert-base-vi-cased/config.json'
init_checkpoint = '/kaggle/working/distilbert-base-vi-cased/pytorch_model.bin'
output_dir = './output_model'

In [15]:
bert_config = BertConfig.from_json_file(bert_config_file)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/104k [00:00<?, ?B/s]

In [16]:
model_state_dict = torch.load(output_model_file)
model = BertForCloze(bert_config, num_choices=7)
model.load_state_dict(model_state_dict)
model.to(device)
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [17]:
context = 'Biết nhẫn nại và kiên trì, luôn luôn có lòng dũng cảm cao hơn kẻ thù, chúng ta sẽ đạt đến những điều tốt đẹp nhất, đó là những điều có giá trị lớn lao và mang tầm quan trọng đặc biệt đối với mỗi một cá nhân và chúng ta đều nhận thức được điều đó. Và câu tục ngữ “[BLANK1]” cho đến hôm nay vẫn luôn là một kinh nghiệm rèn luyện cá nhân vô cùng quý báu.'
choices = [     "Ăn mày nơi cả thể, làm rể nơi nhiều con",
                "Vẽ đường cho hươu chạy",
                "Một miếng khi đói bằng một gói khi no",
                "Ăn thì vóc, học thì hay",
                "Có công mài sắt, có ngày nên kim",
                "Giặc đến nhà, đàn bà cũng đánh",
                "Trẻ trồng na, già trồng chuối"
            ]
answer = 4

In [18]:
ans = get_feature_for_pred(context, choices, is_list = 0, arr = 0)
print(choices[ans])

Có công mài sắt, có ngày nên kim


In [19]:
context = "Từ xưa, ông cha ta đã quan niệm mục đích của việc học trước hết là để làm người, rồi sau đó là tham gia việc làng, việc nước:“Học là học để làm người/ Biết điều hơn thiệt biết lời thị phi”. Câu ca dao này có ngụ ý là muốn khuyên chúng ta phải học tập để trở thành người có ích và nên học những gì tốt chứ không nên học tập những cái xấu. Cho nên, dù khó khăn, gian khổ đến mấy, cha mẹ vẫn cố gắng cho con em mình đi học. Với lối so sánh, ví von mộc mạc, ông cha ta đã khẳng định vai trò quan trọng của việc học tập: “Người mà không học, khác gì đi đêm/ [BLANK3]”; “Ngọc kia chẳng giũa chẳng mài/ Cũng thành vô dụng cũng hoài ngọc đi” - ý nói nếu không học con người sẽ trở nên mù mờ, tăm tối, dẫu có là ngọc đi chăng nữa mà không được mài, được giũa thì ngọc ấy cũng chẳng có giá trị gì. Hay câu ca dao: “Học là học biết giữ giàng/ Biết điều nhân nghĩa biết đàng hiếu trung”để khuyên mỗi người chúng ta cần phải học tập những đạo lý, lễ nghĩa làm người. Hoặc câu:“Người không học, không có sự hiểu biết/ Trẻ mà không học, lớn không làm được việc gì”; “[BLANK2]”,đã cho thấy việc học không có bất cứ giới hạn nào, mà học tập vừa là trách nhiệm, vừa là quyền lợi, nó luôn đòi hỏi sự nỗ lực và quyết tâm cao của mỗi con người. Câu tục ngữ: “[BLANK1]”, ý muốn nhắc nhở chúng ta phải rèn luyện ý chí , nghị lực để vươn lên trong cuộc sống. "
choices = [     "Cú đói ăn con",
                "Luyện mãi thành tài, miệt mài tất giỏi",
                "Đồng tiền đi liền khúc ruột",
                "Muốn biết phải hỏi, muốn giỏi phải học",
                "Người không học như ngọc không mài",
                "trời cũng chiều người",
                "phải trái phân minh"
            ]
answer = [1,3,4]

In [20]:
ans = get_feature_for_pred(context, choices, is_list = 1, arr = 0)
for a in ans:
    print(choices[ans[a]])

Người không học như ngọc không mài
Người không học như ngọc không mài
Người không học như ngọc không mài


In [21]:
context = ["Tuy nhiên, bên cạnh những câu đề cao và ca ngợi đức tính tốt của con người, cũng có những câu thành ngữ phê phán các thói xấu như lười biếng, lười lao động như câu “[BLANK1]”. Câu thành ngữ như hồi chuông cảnh tỉnh con người mau chóng thay đổi tư duy tiêu cực để đưa cá nhân, cộng đồng hoàn thiện và phát triển hơn. Vì trong cuộc sống, giàu có và thành công không dành cho những kẻ lười nhác.",
          "Ở nơi cần tới sự khe khắt, tỉ mẩn và chính xác cao độ mà đem con mắt [BLANK1] ra thì dễ đứt cước. Hàng xóm sống cạnh nhau thì có thể lấy chín bỏ làm mười làm phương châm sống, nhưng khi họp tổ dân phố thì phải loại chín bỏ làm mười ra cho sự nghiêm túc và công minh có đất tồn tại. Nhân viên an ninh hàng không mà dùng con mắt chín bỏ làm mười thì không ổn tí nào."]
choices = [[
            "Nói phải củ cải cũng nghe",
            "Cầm cân nảy mực",
            "Há miệng chờ sung",
            "Đánh rắn phải đánh dập đầu",
            "Ăn hết nhiều, chứ ở hết bao nhiêu",
            "Muốn biết phải hỏi, muốn giỏi phải học",
            "Không có lửa làm sao có khói"
            ],
         [
            "Trẻ cậy cha, già cậy con",
            "Lợn giò, bò bắp",
            "Thứ nhất phạm phòng, thứ nhì lòng lợn",
            "Chín bỏ làm mười",
            "Học một biết mười",
            "Không thầy đố mày làm nên",
            "Hay học thì sang, hay làm thì có"
        ]   ]

answer = [2], [3]

In [22]:
ans = get_feature_for_pred(context, choices, is_list = 1, arr = 1)
for idx, a in enumerate(ans):
    print(choices[idx][ans[a]])
# ans

Không có lửa làm sao có khói
Học một biết mười


In [28]:
context = ["Đây là triết lý giáo dục lấy người thầy làm trung tâm, lời thầy là khuôn vàng thước ngọc\". Tuy nhiên, trong cách hiểu truyền thống thông thường về câu thành ngữ \"[BLANK2]\" thì \"Tôn sư\" không có nghĩa là đặt người thầy vào vị trí trung tâm của của việc truyền giảng. Nếu như thế là ta đã hiểu sang vấn đề kỹ thuật chuyên môn ngành sư phạm. Tôn sư là đề cao vai trò của người thầy, và người thầy luôn ở vị trí được tôn kính. Theo như một câu tục ngữ của người Việt Nam ta: \"[BLANK1]\". Bởi lẽ, trong giáo dục, nói một cách cụ thể là trong việc dạy và học, người học luôn đóng vai trò là trung tâm của sự hội tụ kiến thức. Theo đó, người thầy giáo phải đánh giá đúng đối tượng là người học, hiểu và nắm bắt suy nghĩ của người học, hiểu tâm lý và khả năng nhận thức của người học để từ đó xây dựng nội dung giảng dạy, lựa phương pháp truyền thụ cho thích hợp và định lượng nội dung giảng dạy. Mọi sự đánh giá chủ quan, phiến điện không đúng về đối tượng người học đều dẫn đến thất bại trong giáo dục. \"Tôn sư\" không nghĩa là thầy luôn luôn đúng, vì điều đó còn tùy thuộc vào sức khỏe, trạng thái tâm lý của thầy giáo, sự thay đổi của điều kiện ngoại cảnh mà đã tác động đến hoạt động giáo dục. \"Trọng đạo\" cũng có nghĩa là trọng chân lý, xét trong tình huống giáo dục cụ thể chân lý là của cùng một chủ thể thầy dạy đưa ra, nhưng cũng có khi là chân lý được học trò đúc kết, tích lũy trong hoạt động sống nói chung. Vì thế, trong giáo dục, học trò vẫn có thể tranh luận với thầy, phản biện lại thầy về kiến thức chân lý mà vẫn giữ nguyên đạo lý và sự tôn sư.  Như thế, sự \"tôn sư\" đi liền với \"trọng đạo\" không tách rời nhau mà luôn ở trong cùng một quan niệm nên không hề lỗi thời như quan điểm của tác giả Trương Khắc Trà trong bài viết. Trong bài viết, tác gỉả viết: \"Tiếc thay, chế độ phong kiến - Nho giáo đã hết vai trò lịch sử, đã lùi vào hậu trường nhưng tư tưởng giáo dục của nó vẫn hằn in trong cách nghĩ, cách học, cách làm giáo dục của người Việt trong thế kỷ XXI.   Ngày nay, \"tôn sư trọng đạo\" vẫn mang ý nghĩa tôn vinh người thầy và nghề dạy học, nhưng giáo dục ngày nay cơ bản đã khác xưa, mối quan hệ thầy trò cũng cũng phải vận động sao cho phù hợp với thời cuộc\". Ở đây, cho dù là sự vận động như thế nào đi chăng nữa thì vai trò của người thầy cũng không thể thay thế được. Người Trung Quốc có câu: \"Một gánh sách không bằng một người thầy giỏi\". Mặt khác, sự \"tôn sư\" ở đây còn được hiểu là kính trọng thầy về kiến thức và đạo đức, nhưng cũng còn ý nghĩa khác nữa là quý mến thầy trong cách hiểu về tình người. ",
          "Nhiều người cho rằng từ những thành ngữ trên, người Việt đã phát triển thành câu [BLANK2]. Tuy nhiên, theo chúng tôi, Nhất tự vi sư (一字為師) có khả năng là của Trung Quốc, vì đã từng xuất hiện trong bộ Trần Thị Liên Châu tập (陳氏聯珠集), quyển 10 của Thang Tín Đỗ (năm 1802). Ngoài ra, trong Hán ngữ còn có câu Nhất nhật vi sư, chung thân vi phụ (Một ngày cũng là thầy, suốt đời mới là cha). Chỉ có thành ngữ Bán tự vi sư (半字為師) mới có khả năng là của Việt Nam, kết hợp với thành ngữ Nhất tự vi sư của Trung Quốc để thành Nhất tự vi sư, bán tự vi sư ([BLANK1])."]
choices = [[
                "Nói nhăng nói cuội",
                "Tôn sư trọng đạo",
                "Đoàn kết thì sống, chia rẽ thì chết",
                "Thứ nhất phạm phòng, thứ nhì lòng lợn",
                "Cha sinh chẳng bằng mẹ dưỡng",
                "Há miệng mắc quai",
                "Không thầy đố mày làm nên"
            ],
         [
                "Cá mè một lứa",
                "Trẻ cậy cha, già cậy con",
                "Miệng quan trôn trẻ",
                "Nhất tự vi sư, bán tự vi sư",
                "Một chữ cũng là thầy, nửa chữ cũng là thầy",
                "Thật thà là cha quỷ quái",
                "Sát nhân giả tử"
            ]]

answer = [6, 1], [4, 3]

In [29]:
ans = get_feature_for_pred(context, choices, is_list = 1, arr = 1)
l = []
for c in context:
    l.append(len(re.findall(r'\[BLANK\d+\]', c)))
count = 0
flag = 0
for idx, a in enumerate(ans):
    print(choices[flag][ans[a]])
    count += 1
    if count == l[flag]:
        flag += 1
        count = 0

Tôn sư trọng đạo
Tôn sư trọng đạo
Cá mè một lứa
Cá mè một lứa
